# 生成3Dgs （照片重建）

按照正常步骤的三维高斯重建

In [ ]:
import os
print(f"当前目录: {os.getcwd()}")
if 'arch-gaussian' in os.listdir():
    JUPYTER_ROOT = os.getcwd()
    os.chdir('arch-gaussian')
    print(f"更改后的目录: {os.getcwd()}")
#针对远程jupyter目录的操作， 本地运行可以无视

In [ ]:
import config
# 基础设置
config.scene_name = "Nanjing\Jimingsi" # 输入文件夹名称
config.output_name = "Nanjing\Jimingsi"  # 输出文件夹名称
config.sh_degree = 3  # 0~3， 改为0可以缩小文件大小，但是无法被现有的unity工具识别， 推荐使用默认3
config.epochs = 1000  # 训练轮次， 3000， 7000， 15000， 30000
config.loaded_iter = None # 是否加载已经训练好的轮次，None 或数字
# 高级设置
config.resolution = 1 # 是否压缩图像，1为原尺寸，-1为自动压缩到不大于1600像素， 支持2的次方的数字
config.densify_until_iter = 15000  # 致密化结束的轮次
config.densify_grad_threshold = 0.0002  # 致密化的阈值，越小增加的越快

# 配置结束，更新配置文件
config.update_colmap_args()
config.update_args()

# 打印修改过后的参数
config.print_updated_colmap_args()
config.print_updated_args()


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./src")
import os
import torch
import numpy as np
np.set_printoptions(suppress=True)
from config import args  # 正式导入args



## 创建scene info 与修复

In [ ]:
# 创建scene info
from manager.scene_manager import load_and_fix_scene

# 根据相机姿态，自动估算地面向上向量，并将场景
scene_info = load_and_fix_scene(args)


## 导入相机

In [ ]:
from manager.camera_manager import CameraManager
cm = CameraManager()
train_cameras, test_cameras = cm.create_cameras(args, scene_info)


## 创建gaussian对象

In [ ]:
from manager.gaussian_manager import GaussianManager
gm = GaussianManager(args, scene_info)
print(f"num points: {gm.gaussians.get_xyz.shape[0]}")

In [ ]:
from manager.train_manager import init_snapshot, take_snapshot, SnapshotCameraMode, SnapshotFilenameMode
init_snapshot(0)
def post_socket(**kwargs):
    """完成每一轮训练后的后处理内容"""
    _iteration = kwargs['iteration']
    take_snapshot(cm, gm,
                  _camera_mode=SnapshotCameraMode.SLOW_ROTATE,
                  _slow_ratio=5,
                  _filename_mode=SnapshotFilenameMode.BY_SNAP_COUNT,
                  _folder_name="snapshots",
                  _iteration_gap=100,
                  _first_period_iteration_gap=10,
                  _first_period_end=400,
                  **kwargs)
    _gaussians = kwargs['gaussians']
    if _iteration % 1000 == 0:
        print(_gaussians.get_xyz.shape)

In [ ]:
from manager.train_manager import train, init_output_folder

init_output_folder(args, scene_info)
train(args,scene_info, gm.gaussians, cm.train_cameras, post_socket=post_socket)

## 预览结果

In [ ]:
from manager.gaussian_manager import GaussianManager
gm = GaussianManager(args, gm.gaussians) # 从gaussians创建GaussianManager
cam = cm.pick_camera(0)
image = gm.render(cam, convert_to_pil=True)
image

## Splat 格式转换

In [ ]:
import os
import numpy as np
from plyfile import PlyData

print(f"当前目录: {os.getcwd()}")
if 'arch-gaussian' in os.listdir():
    JUPYTER_ROOT = os.getcwd()
    os.chdir('arch-gaussian')
    print(f"更改后的目录: {os.getcwd()}")

In [ ]:
point_cloud_path = os.path.join(args.model_path, "point_cloud/iteration_{}".format(args.iterations), "point_cloud.ply")
if not os.path.exists(point_cloud_path):
    print("file not found")

In [ ]:
def splat2np(_path):
    with open(_path) as f:
        _b = f.read()
    _dt = np.dtype([
        ('position', np.float32, 3),
        ('scale', np.float32, 3),
        ('RGBA', np.uint8, 4),
        ('IJKL', np.uint8, 4)
    ])
    return np.frombuffer(_b, _dt)

def np2splat(_data, _save_path):
    _data.tofile(_save_path)
    print(f"data saved to {_save_path}")

def ply2np(_ply_path):
    _dt = np.dtype([
        ('position', np.float32, 3),
        ('scale', np.float32, 3),
        ('RGBA', np.uint8, 4),
        ('IJKL', np.uint8, 4)
    ])

    plydata = PlyData.read(_ply_path)
    x = np.array(plydata.elements[0]['x'])
    y = np.array(plydata.elements[0]['y'])
    z = np.array(plydata.elements[0]['z'])
    scale_0 = np.array(plydata.elements[0]['scale_0'])
    scale_1 = np.array(plydata.elements[0]['scale_1'])
    scale_2 = np.array(plydata.elements[0]['scale_2'])
    rot_0 = np.array(plydata.elements[0]['rot_0'])
    rot_1 = np.array(plydata.elements[0]['rot_1'])
    rot_2 = np.array(plydata.elements[0]['rot_2'])
    rot_3 = np.array(plydata.elements[0]['rot_3'])
    r = np.array(plydata.elements[0]['f_dc_0'])
    g = np.array(plydata.elements[0]['f_dc_1'])
    b = np.array(plydata.elements[0]['f_dc_2'])
    a = np.array(plydata.elements[0]['opacity'])

    position = np.stack((x,y,z), axis=1)
    scales= np.stack((scale_0, scale_1, scale_2), axis=1)
    rots = np.stack((rot_0, rot_1, rot_2, rot_3), axis=1)
    rgba = np.stack((r,g,b,a), axis = 1)

    qlen = np.square(rots).sum(axis=1)
    rots = rots / qlen[:, np.newaxis] * 128 + 128
    rots = np.clip(rots, 0, 255)

    scales = np.exp(scales)

    SH_C0 = 0.28209479177387814
    rgba[:,0:3] = (0.5 + SH_C0 * rgba[:,0:3]) * 255
    rgba[:, 3] = (1 / (1 + np.exp(-rgba[:, 3]))) * 255
    rgba = np.clip(rgba, 0, 255)

    rots = rots.astype(np.uint8)
    rgba = rgba.astype(np.uint8)

    merged = np.empty((position.shape[0]),dtype=_dt)

    merged['position'] = position
    merged['scale'] = scales
    merged['RGBA'] = rgba
    merged['IJKL'] = rots

    return merged


In [ ]:
data = ply2np(point_cloud_path)
print(data[0])
save_path = f"{os.path.dirname(point_cloud_path)}/output.splat"
np2splat(data, save_path)

In [ ]:
import http.server
import socketserver

# 指定根目录的位置
root_directory = './web/WebGLViewer'

# 设置服务器的端口号
port = 8000

# 创建一个简单的 HTTP 请求处理器类
Handler = http.server.SimpleHTTPRequestHandler

# 指定根目录的位置
Handler.directory = root_directory

# 创建一个服务器，监听指定的端口
with socketserver.TCPServer(("", port), Handler) as httpd:
    print(f"Serving at port {port}")
    # 启动服务器
    httpd.serve_forever()
